In [85]:
import numpy as np
import re
from typing import List, Dict, Iterator, Pattern
import pandas as pd
from pandas import DataFrame as df

### Reading trace data

In [86]:
# nn, sink_mode,
#

In [87]:
sink_id_dict = {
    "0": 254,
    "1": 253,
    "2": 252
}

def parse_file_records(regex: Pattern, path: str) -> Iterator[Dict]:
    """Yield one dict per matching log line from `path`."""
    with open(path, "r") as f:
        for line in f:
            m = regex.match(line)
            if not m:
                continue

            sink_id = sink_id_dict[m.group("sink_id")] if m.group("sink_id") in sink_id_dict else m.group("sink_id")
            yield {
                "time": float(m.group("time")),
                "sink_id": int(sink_id),
                "packet_id": int(m.group("packet_id")),
                "src_ip": int(m.group("src_ip")),
            }

In [88]:


LOG_PATH = "log_udp.out"
RESULT_PATH = "output.out"

_pattern = re.compile(r"\[(?P<time>\d+(?:\.\d+)?)\]::DBG::UWUDP\((?P<sink_id>\d+)\)::recv\(Packet \*, int\)::new packet with id (?P<packet_id>\d+) from ip (?P<src_ip>\d+) : \d+")

udp_df = df.from_records(parse_file_records(_pattern, LOG_PATH))
result_df = pd.read_csv(RESULT_PATH, skiprows=17, header=0)
result_df = result_df.replace("_", np.nan)
# print("Parsed rows:", len(udp_df))
print(udp_df.head())
print(result_df.head())

if udp_df.empty:
    print("No matching UDP recv lines found.")
    raise Exception()

summary = udp_df.groupby("packet_id") \
    .agg(first_time=("time", "min"), last_time=("time", "max"), duplicates=("packet_id", "count"), sinks=("sink_id", lambda s: sorted(set(s)))) \
    .reset_index()

print(summary.head())

total_recv = len(summary)
total_sent = result_df["n_pkts"].sum()
print("Total received packets:", total_recv)
print("Total transmitted packets: ", total_sent)
print(f"Total PDR: {total_recv / total_sent * 100:.2f}")

      time  sink_id  packet_id  src_ip
0  4.28104      253          0      55
1  4.33242      254          0      55
2  5.01950      253          0      55
3  5.07088      254          0      55
4  5.75796      253          0      55
   ID      x      y  n_pkts  sink1_rcv_pkts  sink1_per  sink1_throughput  \
0   0  63.31  52.55      88              76       0.14              0.14   
1   1  -0.89  33.05      97              84       0.13              0.16   
2   2 -26.19  69.85     104              89       0.14              0.17   
3   3 -79.82  50.64      99              87       0.12              0.16   
4   4  87.81  54.22     100              84       0.16              0.16   

   sink2_rcv_pkts  sink2_per  sink2_throughput  sink3_rcv_pkts  sink3_per  \
0              50       0.43              0.09              78       0.11   
1              80       0.18              0.15              82       0.15   
2              90       0.13              0.17              72       0.30   
3